In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to the activation response .npy files
PROMPT = "A long time ago"
PROMPT_LENGTH = len(PROMPT)

DATA = [
    "data/1M/2_1M_experiment800N_3_TEMPS_PROMPTA_per_trial_tensor.npy",
    "data/8M/8M_experiment800N_9_TEMPS_PROMPTA_per_trial_tensor.npy",
]

# Load in data and create the means. 
DATA = [np.load(d) for d in DATA]
AVERAGE_RESPONSES = [np.mean(d, 2) for d in DATA]

# Instantiate the tokenizer and tokenize input prompt. 
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
input_ids = tokenizer(PROMPT, return_tensors="pt").input_ids


model = AutoModelForCausalLM.from_pretrained('/Users/joshstallings/Desktop/LipshutzLab/TinyStories-1M')
model.eval()


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlock(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=64, out_features=64, bias=False)
            (v_proj): Linear(in_features=64, out_features=64, bias=False)
            (q_proj): Linear(in_features=64, out_features=64, bias=False)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_features=256, out_

#### Output for the mean trajectories


In [16]:
temp_idx = 1
seed_idx = 2
tensor = torch.from_numpy(AVERAGE_RESPONSES[0][temp_idx][seed_idx])
tensor = tensor.float()
x1 = model.transformer.ln_f(tensor)  # Apply normalization layer
x2 = model.lm_head(x1)  # Send through language model head


to_decode = torch.argmax(x2, dim=-1)
to_decode[:len(input_ids[0])] = input_ids
decoded_text = tokenizer.decode(to_decode.tolist())
decoded_text


'A long time ago there was a little, named Lily. She loved to play and the the.. day,'

#### Output from the actual trajectories

In [24]:
experiment_idx = 500
tensor = torch.from_numpy(DATA[0][temp_idx][seed_idx][experiment_idx])
tensor = tensor.float()
x1 = model.transformer.ln_f(tensor)  # Apply normalization layer
x2 = model.lm_head(x1)  # Send through language model head


to_decode = torch.argmax(x2, dim=-1)
to_decode[:len(input_ids[0])] = input_ids
decoded_text = tokenizer.decode(to_decode.tolist())
decoded_text

'A long time ago was a little named Timmy. Tim loved very excited to go with his toy. the park'